### 1) Preprocess text using Python's Natural Language Toolkit
### 2) Produce topic modelling with Latent Dirichlet Allocation
### 3) Visualize intertopic distance

In [233]:
import pickle

from pprint import pprint
import pandas as pd
import string
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [462]:
def get_wordnet_pos(treebank_tag):
    """Convert the part-of-speech naming scheme
       from the nltk default to that which is
       recognized by the WordNet lemmatizer"""

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

    
def preprocess_series_text(data):
    """Perform complete preprocessing on a Pandas series
       including removal of alpha numerical words, normalization,
       punctuation removal, tokenization, stop word removal, 
       and lemmatization."""
    
    # remove alpha numerical words and make lowercase
    alphanum_re = re.compile(r"""\w*\d\w*""")
    alphanum_lambda = lambda x: alphanum_re.sub('', x.strip().lower())

    data = data.map(alphanum_lambda)

    # remove punctuation
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    punc_lambda = lambda x: punc_re.sub(' ', x)

    data = data.map(punc_lambda)

    # tokenize words
    data = data.map(word_tokenize)

    # remove stop words
    sw = stopwords.words('english')
    sw += ['ve', 'thank', 'don', 'just', 'did', 'lot', 'let', 'doesn', '–', '—', 'chris', 'anderson', 'yeah', 'ca', 'oh', 'mike', 'also', 'ok', 'percent']
    sw = set(sw)
    sw_lambda = lambda x: list(filter(lambda y: y not in sw, x))

    data = data.map(sw_lambda)

    # part of speech tagging--must convert to format used by lemmatizer
    data = data.map(nltk.pos_tag)
    pos_lambda = lambda x: [(y[0], get_wordnet_pos(y[1])) for y in x]
    data = data.map(pos_lambda)

    # lemmatization
    lemmatizer = WordNetLemmatizer()
    lem_lambda = lambda x: [lemmatizer.lemmatize(*y) for y in x]
    data = data.map(lem_lambda)
    
    return data

In [463]:
with open('database_2455_1.pkl', 'rb') as picklefile:
    db_raw = pickle.load(picklefile)

In [525]:
df = pd.DataFrame(db_raw)


In [465]:
def join_fragments(row):
    t = ' '.join(row['text'])
    return t

In [466]:
df['text'] = df.apply(join_fragments, axis=1)

In [467]:
df = df.replace('', np.nan).dropna()

In [3]:
# df.sort_values(by='views', ascending=False)

In [469]:
df['tokens'] = preprocess_series_text(df.text)

In [471]:
import datetime as dt
df['date'] = df['date'].apply(lambda x: dt.datetime(x[1], x[0], 15))
                                

In [2]:
# corpus = []
# for item in df.tokens:
#     corpus.append(' '.join(item)) #final corpus list
    
# corpus

In [474]:
df['corpus'] = corpus

### Final dataframe with original text and tokenized corpus in last column

In [475]:
df.head()

,date,info,seconds,speaker,text,title,url,views,tokens,corpus
0,2016-11-15,Sleep researcher,633.0,Wendy Troxel,"It's six o'clock in the morning, pitch black o...",Why school should start later for teens,/talks/wendy_troxel_why_school_should_start_la...,106879.0,"[six, clock, morning, pitch, black, outside, y...",six clock morning pitch black outside year old...
1,2017-04-15,"Policy entrepreneur, climate expert, author",787.0,Ted Halstead,I have a two-year-old daughter named Naya who ...,A climate solution where all sides can win,/talks/ted_halstead_a_climate_solution_where_a...,115611.0,"[two, year, old, daughter, name, naya, mistake...",two year old daughter name naya mistake impres...
2,2016-11-15,Caregiver,969.0,Lucy Kalanithi,A few days after my husband Paul was diagnosed...,What makes life worth living in the face of death,/talks/lucy_kalanithi_what_makes_life_worth_li...,385291.0,"[day, husband, paul, diagnose, stage, iv, lung...",day husband paul diagnose stage iv lung cancer...
3,2017-04-15,AI expert,1055.0,Stuart Russell,This is Lee Sedol. Lee Sedol is one of the wor...,3 principles for creating safer AI,/talks/stuart_russell_how_ai_might_make_us_bet...,304731.0,"[lee, sedol, lee, sedol, one, world, great, go...",lee sedol lee sedol one world great go player ...
4,2017-04-15,"Actor, producer, activist",1071.0,Shah Rukh Khan,"Namaskar. I'm a movie star, I'm 51 years of ag...","Thoughts on humanity, fame and love",/talks/shah_rukh_khan_thoughts_on_humanity_fam...,2232142.0,"[namaskar, movie, star, year, age, use, botox,...",namaskar movie star year age use botox yet cle...


In [476]:
# with open('corpus_final.pkl', 'wb') as picklefile:
#     pickle.dump(df, picklefile)

In [342]:
count_vectorizer = CountVectorizer(ngram_range=(1,1), max_df=0.65, min_df=.01, 
                                   token_pattern="\\b[a-z][a-z]+\\b")

X = count_vectorizer.fit_transform(corpus)

In [343]:
X.shape

(2300, 5533)

In [478]:
n_topics = 25
n_iter = 10

lda = LatentDirichletAllocation(n_topics=n_topics,
                                max_iter=n_iter,
                                random_state=0)

lda.fit(X)

/Users/brianturner/anaconda/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_jobs=1, n_topics=25, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [460]:
# from sklearn.externals import joblib
# joblib.dump(lda, 'lda_model_2_20_topics_65_df.pkl') 

['lda_model_2_20_topics_65_df.pkl']

In [479]:
def print_topic_top_words(model, cv, n_top_words=10):
    feature_names = cv.get_feature_names()
    
    for topic_vec in enumerate(model.components_):
        topic_num = topic_vec[0]
        topic_words = topic_vec[1]
        
        print('Topic {}:'.format(topic_num + 1))
        
        topic_values = sorted(zip(topic_words, feature_names), 
                              reverse=True)[:n_top_words]
    
        print(' '.join([y for x,y in topic_values]))
        # pprint(topic_values)
        
    return

#### top 20 words in 25 topics:

In [480]:
n_top_words = 20
print_topic_top_words(lda, count_vectorizer, n_top_words)

Topic 1:
ocean fish sea water ice place big foot particle animal long deep area earth coral last meter high mile whale
Topic 2:
woman child men girl story family mother man love young father school old never home community ask boy parent meet
Topic 3:
woman child disease never black able problem name family doctor mother allow health story love human government body still create
Topic 4:
war country state political power american democracy government group medium vote become attack united religion china election muslim kill middle
Topic 5:
cord spinal soap opera loop nervous initiate stimulate coordinate ego descend starbucks obstacle lesson robot segment stimulation big walk fight
Topic 6:
black hour image data girl hole become woman social help mother job ask often piece white together record human week
Topic 7:
kid play school music learn student game video teacher sound teach child education ask great idea help hear share old
Topic 8:
child feel never hour minute month four help ev

### Visualize intertopic distance

In [348]:
import pyLDAvis, pyLDAvis.sklearn
from IPython.display import display

# Setup to run in Jupyter notebook
pyLDAvis.enable_notebook()

# Create the visualization
vis = pyLDAvis.sklearn.prepare(lda, X, count_vectorizer)

# Export as a standalone HTML web page
pyLDAvis.save_html(vis, 'lda_2_65_df.html')

# Let's view it!
display(vis)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
11     15.354370        1       1 -0.120195 -0.038263
9      13.841625        1       2 -0.126860 -0.014057
18     11.650252        1       3 -0.103831  0.029084
6      11.488763        1       4 -0.122573  0.008678
1       9.223745        1       5 -0.111001 -0.073174
7       8.729577        1       6 -0.118952 -0.061900
14      6.928551        1       7 -0.102556  0.011036
0       5.754985        1       8 -0.117197  0.072703
13      5.045006        1       9 -0.105751  0.102243
19      4.475241        1      10 -0.092904 -0.025475
3       4.353818        1      11 -0.096349 -0.041948
15      0.718460        1      12 -0.025576 -0.002803
17      0.716748        1      13  0.180771  0.359038
16      0.671698        1      14  0.045455  0.212390
10      0.366234        1      15  0.270798 -0.017949
8       0.247803        1      16  0.276461 -0.101576
2       0.211265        1      17  0.258403 -0.131274
12      0.205811        1      18  0.242657 -0.061649
5       0.008023        1      19 -0.015362 -0.112572
4       0.008023        1      20 -0.015437 -0.112532, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
810   Default  3936.000000        child  3936.000000  30.0000  30.0000
2575  Default  1975.000000  information  1975.000000  29.0000  29.0000
551   Default  2112.000000         body  2112.000000  28.0000  28.0000
3395  Default  2680.000000       number  2680.000000  27.0000  27.0000
2208  Default  1701.000000   government  1701.000000  26.0000  26.0000
3255  Default  2906.000000         move  2906.000000  25.0000  25.0000
5478  Default  3999.000000        woman  3999.000000  24.0000  24.0000
1887  Default  2173.000000       family  2173.000000  23.0000  23.0000
1146  Default  3732.000000      country  3732.000000  22.0000  22.0000
2908  Default  2105.000000        light  2105.000000  21.0000  21.0000
2292  Default  2369.000000         hand  2369.000000  20.0000  20.0000
584   Default  2933.000000        brain  2933.000000  19.0000  19.0000
1084  Default  1530.000000      control  1530.000000  18.0000  18.0000
4615  Default  2192.000000        space  2192.000000  17.0000  17.0000
3170  Default  2774.000000      million  2774.000000  16.0000  16.0000
3573  Default  1093.000000       parent  1093.000000  15.0000  15.0000
1557  Default  1678.000000        earth  1678.000000  14.0000  14.0000
2     Default  2781.000000         able  2781.000000  13.0000  13.0000
5373  Default  1741.000000         walk  1741.000000  12.0000  12.0000
2486  Default  1573.000000        image  1573.000000  11.0000  11.0000
5397  Default  2537.000000        water  2537.000000  10.0000  10.0000
5234  Default  1083.000000     universe  1083.000000   9.0000   9.0000
815   Default   843.000000        china   843.000000   8.0000   8.0000
1436  Default  1326.000000      disease  1326.000000   7.0000   7.0000
4235  Default   907.000000        robot   907.000000   6.0000   6.0000
522   Default  1194.000000        black  1194.000000   5.0000   5.0000
3117  Default  1584.000000          men  1584.000000   4.0000   4.0000
727   Default  1863.000000         case  1863.000000   3.0000   3.0000
2047  Default  2262.000000         four  2262.000000   2.0000   2.0000
2920  Default  1577.000000         line  1577.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
359   Topic20     0.020888      augment    64.538956   1.3947  -8.6101
3688  Topic20     0.020704       picked    26.830549   2.2636  -8.6189
2774  Topic20     0.020879      jupiter    63.046073   1.4177  -8.6105
3766  Topic20     0.020692     populate    26.314388   2.2824  -8.6195
5478  Topic20     0.021627        woman  3999.498649  -2.6972  -8.5753
1557  Topic20     0.021495        earth  1678.083811  -1.8348  -8.5814


### Turn count-vectorized matrix to an array of the top 100 words

In [428]:
X.toarray()

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 2, 0, 0]], dtype=int64)

In [448]:
df_cv = pd.DataFrame(X.toarray(),columns=count_vectorizer.get_feature_names()).T

In [450]:
df_cv['total'] = df_cv.sum(axis=1)

In [523]:
df_total = df_cv['total'].sort_values(ascending=False)
df_total[:100] # top 100 words

human          4360
idea           4281
problem        4251
great          4138
big            4095
woman          3984
ask            3862
place          3818
story          3776
country        3735
create         3702
become         3670
percent        3666
today          3662
system         3650
child          3561
question       3561
build          3475
fact           3454
learn          3400
last           3204
end            3178
another        3177
turn           3165
never          3139
technology     3092
old            3078
help           3077
long           3032
love           3023
               ... 
four           2233
person         2217
may            2212
space          2192
experience     2192
grow           2188
family         2183
bring          2175
run            2158
whole          2144
away           2137
home           2131
power          2126
everything     2117
body           2112
light          2095
bad            2082
five           2058
cell           2046
